<a href="https://colab.research.google.com/github/gisalgs/notebooks/blob/main/projections1-colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Map Projections (Halloween Edition 2021)

<img width=555px src="figures/halloween-hall-of-fame.png"/>

The goal of a map projection is to transform points in a spherical coordinate system onto a two dimensional Cartesian coordinate system. In other words, points on earth surface that are measured in longitude ($\lambda$) and latitude ($\varphi$) pairs become points on a two dimensional plane measured as horizontal ($x$) and vertical ($y$) coordinates. We can imagine there are functions that takes in the longitude and latitude of a point and returns the X and Y coordinates:

$x = f(\lambda, \varphi)$  
$y = f(\lambda, \varphi)$

A straightforward transformation from a mathematical perspective is the special case of the **equirectangular** projection that basically does not do anything other than just use the longitude as the X coordinate, and latitude as Y:

$x = \lambda$  
$y = \varphi$

## Data preparation

To see how any projection actually looks like, we will need the appropriate data sets to begin with. Specifically we need two kinds of data here, the world coast lines and the graticule:

* The world coast lines from Natural Earth, in two formats:
    * A shapefile that is compressed as a tar.gz file at the [Github site](https://github.com/gisalgs/data/blob/master/ne_110m_coastline.tar.gz) (need to uncompress it after downloading).
    * A GeoJSON file at the [Github site](https://github.com/gisalgs/data/blob/master/ne_110m_coastline.geojson). Click on the Raw button to save the file or directly download it at https://raw.githubusercontent.com/gisalgs/data/master/ne_110m_coastline.geojson.
    * Both file formats can be useful on local computers. However, the GeoJSON is also suitable for online notebooks and can be used without being downloaded to local computers as well. This is also a place to re-enforce the difference between the two formats and appreciate why GeoJSON has picked so much popularity lately. Clicking on [this link](https://github.com/gisalgs/data/blob/master/ne_110m_coastline.geojson) allows us to see immediately rendering of the spatial data in the GeoJSON file, which is made possible by a lot of reasons but first reason being everything in the spatial data is encoded in ONE place/file and in plain text. This enables the file to be read and processed easily in different programming languages. [Here](https://github.com/gisalgs/data/blob/master/uscnty48area.geojson) is another example of GeoJSON using the counties data. But shapefile, unfortunately, is not designed in this way, which is understandable given how old this one is, because it has multiples file encoded in binary code that must be specially decoded.
* The graticule, however, is not stored in a file (though we could certainly do that). Instead, we will dynamically generate the graticule using Python code. 

### Generating the graticule

> The term graticule is commonly used in cartography, referring to the network formed by the lines of latitudes and longitudes, or parallels and meridians. Sometimes people use the term grid for this purpose, but we as geospatial professionals should know better: lines that form a grid are not necessarily parallels and meridians. We can use any lines to make up a grid.

We first find a way to systematically generate the lines of latitudes and longitudes. Before we get to the coding part, let's decide how we want our data to be organized. While there are many ways to do this, the bottom line for an projection is that we must be able to change the coordinates of each point from one system to another. There is not way this can be avoid. For this reason, we will literally store each point individually. Along with the point's coordinates, we also give it a unique number that indicates which line this point belongs to. For example, we make sure `[12, -83, 40]` is the point that is on a line with an ID of 12, and the coordinates of the point is -83, 40 (sounds familiar?). We will then put all the points in this way, in a list.

Now we are going to get the points on the graticule first. We can't get all the points because that wold be infinite number of them. Instead, we use an interval of 10 for the points on the parallels and meridians. Now we have the code:

In [ ]:
raw_points=[]
linenum = 0
lat0, lat1 = -90, 91    # start and end lagitidues (south and north poles)
lon0, lon1 = -180, 181  # range of longitudes

# points on the parallels
for lat in range(lat0, lat1, 10):
    for lon in range(lon0, lon1, 10):
        raw_points.append([linenum, lon, lat])
    linenum += 1

# TODO:
#     use the same logic to get the points on the meridians








numgraticule = linenum

The last two points in the graticule, as we set it, will be the two points on the meridian 180 for latitudes 80 and 90. 

In [ ]:
raw_points[-2], raw_points[-1]

Now we check how many lines form the graticule. 

In [ ]:
numgraticule

### Getting the coastlines

In order to directly load the online GeoJSON file into our program here, we need to import a couple of modules. The first module (urllib) is a powerful tool designed to scrap information from online sources. The second is the json module that provides tremendous convenience to handle JSON data in general in Python programs.

In [ ]:
import urllib.request as request
import json 

The next step is to read the data from the online file. We will use the `with` statement in Python. When we read data from a file (online or not), something special happens: the bits in the file will be continuously read from the file into the computer memory. We don't call it streaming, but it is close. During the process, we want to make sure the file keeps open so we can continue to read (or write if applicable) the stuff. The `with` statement does just that: it makes sure we can continue to hold the file for reading and writing, and it will release the resource (i.e., file) when we are done. In Python, a `with` statement uses a colon to specific the code that will handle the file. When we exit the code block (i.e., un-indent) the resource is released, meaning other programs can use it. 

The use of `request.urlopen` returns an object of a class called [HTTPResponse](https://docs.python.org/3.3/library/http.client.html#httpresponse-objects) that gives us a handle to deal with the HTTP response from an online service. HTTP stands for Hypertext Transfer Protocol, a foundation on which the entire Web is built. For our case, it allows us to read data from the online source. 

When we read things from a web page (note the GeoJSON file is treated as a web page as everything on the web because of its URL), we get a string. The easiest and quickest way to convert a string into an actual JSON object or Python dictionary is to use the `loads` method from the `json` module. This directly returns an object that can be immediately used -- it is a dictionary.

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/master/ne_110m_coastline.geojson'
with request.urlopen(url) as response:
    coastline_obj = json.loads(response.read())

The keys in this dictionary includes the ones we have seen before.

In [ ]:
coastline_obj.keys()

We can see how many features in the GeoJSON:

In [ ]:
len(coastline_obj['features'])

There are 134 lines in the file and we can get each of the features like this:

In [ ]:
f = coastline_obj['features'][0]
f['properties']

Based on this, we can retrieve every point from each of the features and continue to create the list of `[ID, X, Y]` like we did for the lines in the graticule, and append them to the list `raw_points`:

In [ ]:
for f in coastline_obj['features']:
    geom = f['geometry']['coordinates']
    for p in geom[0]:
        raw_points.append([linenum, p[0], p[1]])
    linenum += 1
    
numline = linenum

At the end, putting everything together, there should be 6534 points and 190 lines, where 56 lines are in the graticule and the 134 lines from the coastlines.

In [ ]:
print(len(raw_points), numgraticule, linenum)
print(raw_points[0])
print(raw_points[3000])

### Drawing the world map

With everything we have so far, we can draw a world map. Note what we have here in coordinates are latitudes and longitudes, and we normally refer to such a map or data as "unprojected." We will have more discussion on this later. Right now, we can treat them as the "original" data that haven't been projected to a Cartesian coordinate system with meaningful distance. The "meaningful distance" part is critical here: distances should not be just numbers, but useful units as well so we know from one point to another is how many miles, kilometers, or other units. Using lat and long cannot give us such meaning to attach to a number for a distance measure.

Nevertheless, we have the coordinates and they are numbers, which are as good as any other numbers in terms of drawing them. In order to make it more convenient, we write a function to draw the points:

In [ ]:
def plot_world(ax, points, numgraticule, numline, color=None):
    '''
    Uses the data prepared by worlmap.py to plot the world map.
    
    Input: 
        ax:             matplotlib axes
        points:         a list of [ [ID, X, Y], [ID, X, Y], ...]
        numgraticule:   the number of lines forming the graticule
        numline:        the total number of line IDs
        color:          if not None, the color used to draw coastlines (default: #5a5a5a)

    The user needs to import matplotlib.pyplot first and run plt.show() after calling this function.
    '''

    for i in range(numline):
        if i<numgraticule:
            col = 'lightgrey'
        else:
            col = '#5a5a5a'
            if color is not None:
                col = color
        pts = [[p[1], p[2]] for p in points if p[0]==i]
        l = plt.Polygon(pts, color=col, fill=False, closed=False)
        ax.add_line(l)

    ax.axis('equal')                       # x and y one the same scale
    ax.axes.get_xaxis().set_visible(False)  # don't show axis
    ax.axes.get_yaxis().set_visible(False)  # don't show axis
    ax.set_frame_on(False)                  # no frame either

The function takes the matplotlib axes and three more required parameters we have prepared already: all the points in a list, the number of lines in the graticule, and the total number of lines. We also have an option to specify the color for the coastlines but we can just take the default (that is some kind of dark grey). We need to import the matplotlib.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
_, ax = plt.subplots(1, 1)
plot_world(ax, raw_points, numgraticule, numline)
plt.show()

## Github modules

We have all the above code wrapped up in a could of github modules. 

- geom.worldmap: the module of https://github.com/gisalgs/geom/blob/master/worldmap.py includes everything we have to prepare the data. Inside this module there is a function called `prep_projection_data` that can be used with four options:
    - `prep_projection_data(fname)` is the default option where `fname` is a the full path to a shapefile that is stored locally. This options uses the `shapex` module as we have seen in the previous week.
    - `prep_projection_data(fname, _use_lib='GEOJSON')` uses a local GeoJSON file where `fanme` is the full path to that file.
    - `prep_projection_data(fname, _use_lib='URL')` uses an online GeoJSON file where fname is the full URL to that file (as we did above).
    - There is a fourth option that uses the OGR library. This requires the GDAL package to be installed. GDAL is a powerful library that supports many vector and raster data formats. It also powers popular GIS software packages such as QGIS. Installing it, however, can be a challenge for different operating systems. We do not use it in this class for that reason. 


Below we show how this works for the online GeoJSON file. But again the code works for local files as well. To use it locally, we should follow what we did with other modules such as point.py: we will put [worldmap.py](https://github.com/gisalgs/geom/blob/master/worldmap.py) in the same **geom** folder as we have done in the previous exercises. 

To use it directly in the notebook (in colab for example), we need to make sure the modules in the geom folder are all updated, as the content of the files may have changed. This is done by forcefully removing the geom folder (line 1 below) and the re-clone the package from the github source (line 2):

In [ ]:
!rm geom -rf
!git clone https://github.com/gisalgs/geom.git 

Now we can import the worldmap module from the geom package:

In [ ]:
from geom.worldmap import *

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/master/ne_110m_coastline.geojson'
raw_points, numgraticule, numline = prep_projection_data(url, _use_lib='URL')
print(len(raw_points))
print(numgraticule, numline)
print(raw_points[0])
print(raw_points[3000])

The `plot_world` function is also included in another module called [plot_worldmap.py](https://github.com/gisalgs/geom/blob/master/plot_worldmap.py) that should also be ready to import now:

In [ ]:
from geom.plot_worldmap import *

In [ ]:
_, ax = plt.subplots(1, 1)
plot_world(ax, raw_points, numgraticule, numline)
plt.show()

## Equirectangular projection

The equirectangular projection can be formulated as follows:

$x = \lambda \cos(\varphi_0)$  
$y = \varphi$

where $\varphi_0$ is the standard latitude or standard parallel that determines the overall width of the projected map. The special case we just saw at the beginning of today's note can be obtained by simply setting $\varphi_0$ to 0 (the equator). 

This all looks straightforward. But there is a caveat here. It should be noted that the lat/lon values we use are typically in **degrees** (as in 0 to 360 degrees for a full circle), but the trigonometric functions in Python will use **radians** (as in 0 to 2$\pi$) as the unit for angles, where 180 degree is equivalent to $\pi$. We will need to convert them, which can be done by importing the function called `radians` from the `math` module.

In [ ]:
from math import cos, radians, pi
print(cos(pi/2.0))
print(pi)
print(radians(360))
print(radians(180))
print(radians(45))
print(cos(radians(90)))
print(cos(radians(360)))

The Python code to make the above transformation for the equirectangular projection is listed below. 

```python
def transform_equirectangular(lon, lat, lat0=0):
    x = lon * cos(radians(lat0))
    y = lat
    return x, y
```

The complete code of this is also stored at gisalgs Github in [transforms.py](https://github.com/gisalgs/geom/blob/master/transforms.py). This python file also contains another function that will soon be used. We will again download this and save it into our **geom** folder. 

We can test this transformation function using the lon and lat of Columbus as an example.

In [ ]:
from geom.transforms import transform_equirectangular
print(transform_equirectangular(-83, 40))

Now we project the original data (in `raw_points`) into a new projection. This has to be done point by point:

In [ ]:
from geom.transforms import transform_equirectangular

points=[]
for p in raw_points:
    p1 = transform_equirectangular(p[1], p[2])
    points.append([p[0], p1[0], p1[1]])

Drawing the new points is simple. Note that we use `points` instead of `raw_points` below:

In [ ]:
_, ax = plt.subplots(1, 1)
plot_world(ax, points, numgraticule, numline)
plt.show()

So this is what happens: the plotting of the "raw" data (i.e., lat/lon) looks exactly the same as the projected data. This shouldn't be a surprise: the projected coordinates are indeed as same as the raw data in the special case when $\phi_0$ is set to 0, which is the default value of the transformation function `transform_equirectangular`. Given how the projection looks, there is a good reason to call this **equirectangular** projection. The equirectangular projection apparently tends to **distort** the area, especially when we move always from the standard parallels. It should be noted that the equirectangular projection does not always produce squires as shown above (see questions below for more detail).


In [ ]:
# TODO:
#   Project the world map using the transform_equirectangular function where lat0 is set to 60.
#   code here:






In [ ]:
# TODO
#    explain below why the above map is only a half of the width as when `lat0` is the default value?
#    You can use write your explanation below in the docstring:






## Sinusoidal projection

There are projections that do not distort size and the [**sinusoidal** projection](https://en.wikipedia.org/wiki/Sinusoidal_projection) is one of them. The equations for the sinusoidal projection are also simple:

$x = (\lambda - \lambda_0) \cos(\varphi)$<br/>
$y = \varphi$

where $\lambda_0$ is central meridian. Because the use of $\cos(\varphi)$, the length of parallels will become zero when $\varphi$ reaches 90 (poles), which forces the convergence of the meridians and therefore preserve the area. Here is the Python function that does the transformation:

```python
def transform_sinusoidal(lon, lat, lon0=0):
    lon1 = lon-lon0
    x = lon1 * cos(radians(lat))
    y = lat
    return x, y
```

Now we follow a similar procedure as did in the previous project to draw the output. The following is the figure that illustrates the sinusoidal projection with prime meridian as the standard meridian. 

In [ ]:
from geom.transforms import transform_sinusoidal

points=[]

# TODO:
#   Use function transform_sinusoidal to project all points in raw_points
#   Append the projected points and their line ID's to list point
#   code here:




In [ ]:
# TODO:
#    Draw world map in Sinusoidal projection
#    code ehre



### Problems with central meridian

What happens if we set `lon0` to 60 using our code? It will yield a strange, asymmetrical map. Why? We are not going to answer this in class and this will be one of the questions in the second homework. 

In [ ]:
points1=[]
for p in raw_points:
    p1 = transform_sinusoidal(p[1], p[2], 60) # central meridian at 60
    points1.append([p[0], p1[0], p1[1]])

_, ax = plt.subplots(1, 1)
plot_world(ax, points1, numgraticule, numline)
plt.show()

### More on equal-area (Tissot's indicatrix)

So how do we know the the Sinusoidal projection is equal-area? There are a lot of signs we can tell. For example, we can examine how the graticule looks. There are patterns that an equal area projection should follow (as discussed in many cartography classes) and we can use those to make some educated conclusions. We can also use math to tell. For a project based on a sphere with a unit radius, an equal area projection will have the following property (Snyder 1987, p. 28):

$$
\frac{\partial x}{\partial \lambda} \frac{\partial y}{\partial \varphi} - \frac{\partial x}{\partial \varphi} \frac{\partial y}{\partial \lambda} = \cos\varphi
$$

where $\lambda$ is longitude and $\varphi$ is latitude.

For those who are comfortable with math, we can definitely conclude that the projection is indeed equal area. But what else? We can actually do some more visualization using a device called [Tissot's indicatrix](https://en.wikipedia.org/wiki/Tissot%27s_indicatrix). The idea is to put some infinitely small circles of the same size (mathematically unit size) at different places on the globe and then see how they look after they are projected. The original tool is a mathematically tool (who else would be interested in circles that are infinitely small?), but we can definitely relax it a little bit by using some circles that are more legible. There is a GeoJSON file that contains some circles at our [github geom repo](https://github.com/gisalgs/data/blob/master/tissot.geojson) and we will see how to use it here.

In [ ]:
url = 'https://raw.githubusercontent.com/gisalgs/data/master/tissot.geojson'
with request.urlopen(url) as response:
    tissots = json.loads(response.read())

Each circle is just a set of systematically generated points that are separated with the same distance on the circle:

In [ ]:
len(tissots['features'][0]['geometry']['coordinates'][0])

We will first put the points on the circle in the same way as we treat the coastlines, but we will start from a new list:

In [ ]:
tissots_raw_points = []

tline = 0
for f in tissots['features']:
    geom = f['geometry']['coordinates']
    for p in geom[0]:
        tissots_raw_points.append([tline, p[0], p[1]])
    tline += 1
    
tissots_points = []

for p in tissots_raw_points:
    p1 = transform_sinusoidal(p[1], p[2])
    tissots_points.append([p[0], p1[0], p1[1]])



The following code calls the `plot_world` function first and then add the new lines (circles) onto the same ax. This way, we can draw everything together in one map.We should see all the circles are rendered at relatively the same size, an indication of an equal area projection.

In [ ]:
_, ax = plt.subplots(1, 1)
plot_world(ax, points, numgraticule, numline, 'darkgrey')

for i in range(tline):
    col = '#fa5a5a'
    pts = [[p[1], p[2]] for p in tissots_points if p[0]==i]
    l = plt.Polygon(pts, color=col, fill=False, closed=False)
    ax.add_line(l)
    
plt.show()

If we have some hard time to tell whether the sizes are the same (the are very close though), we can actually calculate the sizes using what we have learned before and print out the sizes. We just need to import a couple of modules first:

In [ ]:
from geom.point import *
from geom.centroid import *

In [ ]:
areas = []
for f in tissots['features']:
    polygon = []
    for p in f['geometry']['coordinates'][0]:
        p1 = transform_sinusoidal(p[0], p[1])
        polygon.append(Point(p1[0], p1[1]))
    area, _ = centroid(polygon)
    areas.append(area)
    
# TODO:
#   Draw the histogram of the areas of Tissot's indicatrix




So the numbers are not exactly the same, but they are awfully close. The reason for the slight differences is the circles are not small enough. 

## Case study

Let's design a simple but interesting projection.

In [ ]:
def transformx(lon, lat):
    x = lon*(90-lat)/90
    y = lat
    return x, y    

In [ ]:
# TODO
#   show the world map using the projection implemented in function transformx
#   code ehre:





This is very promising in the sense that we have got a project where all the meridian converge at one of the poles. Can we change the formula so that they converge to both poles so that the part of meridian in the southern hemisphere converge to the south pole?

In [ ]:
# TODO
#    change (redefine) the transformx function below so that the meridians converge to both poles




In [ ]:
# TODO
#   project raw_points using the new transformx function



In [ ]:
    
# TODO
#    Draw the world map using newly projected points




Is this an equal area projection? We will once again use the Tissot's indicatrix to visually answer it.

In [ ]:
_, ax = plt.subplots(1, 1)
plot_world(ax, points, numgraticule, numline)

   
tissots_points = []

for p in tissots_raw_points:
    p1 = transformx(p[1], p[2])
    tissots_points.append([p[0], p1[0], p1[1]])


for i in range(tline):
    col = '#fa5a5a'
    pts = [[p[1], p[2]] for p in tissots_points if p[0]==i]
    l = plt.Polygon(pts, color=col, fill=False, closed=False)
    ax.add_line(l)
    
plt.show()

We should be able to see the transformed "circles" (they don't look like circles anymore) have different sizes. However, if we are not sure, we can always calculate the areas of all the transformed circles:

In [ ]:
# TODO:
#   Calculate the areas of the circles projected using the new transformx function
#   Code here



